In [6]:
## Importing Libraries

# Python-based Libraries
from datetime import timedelta, datetime
from pathlib import Path
import os

# Data Analysis Libraries
import numpy as np
import pandas as pd

# Graph plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mplcols
from matplotlib import cm
import folium
import branca.colormap as bcm
    
# Extract CTD
import ctd

In [7]:
class CTD:
    
    def __init__(self, dataName, folderName, ctdPath):
        self.dataName = dataName
        self.folderName = folderName
        self.ctdPath = ctdPath

    def extractData(self): #Extracting the data from the CTD file
        #%%
        self.dataName = {}
        self.folderName = f"{path}/{self.ctdPath}"

        data_columns = ["Pressure", "Temperature", "Salinity", "Oxygen"]

        for ctd_file in os.listdir(self.folderName):
            if ctd_file.endswith(".cnv"):
                data_key = ctd_file.split("_")[0][-1]
                fetched_data = ctd.from_cnv(f'{self.folderName}/{ctd_file}')
                down_data, up_data = fetched_data.split()
                data_value = {"all":fetched_data, "down":down_data, "up":up_data}

                for df_value_key in data_value:
                    data_value[df_value_key].reset_index(inplace=True) # Removing "Pressure" as the index [now a column].
                    data_value[df_value_key] = data_value[df_value_key].rename({"Pressure [dbar]":"Pressure", 
                                                                                "t090C":"Temperature", "sal00":"Salinity", 
                                                                                "sbeox0V":"Oxygen"}, axis = "columns")
                    data_value[df_value_key] = data_value[df_value_key][ data_value[df_value_key]["flag"]==False ]
                    
                    if df_value_key in ["up", "all"]:
                        # Removing data where the pressure is negative: negative pressure indicates 
                        # that the reading is taken outside the sea
                        data_value[df_value_key] = data_value[df_value_key][ data_value[df_value_key]["Pressure"]>=0 ]
                    
                    # Taking only the needed columns
                    data_value[df_value_key] = data_value[df_value_key][data_columns]

                self.dataName[data_key] = data_value

    #Plotting Pressure against Temperature, Salinity and Oxygen _to see the thermocline, halocline and oxycline_.
    def variationPlot(self, filter): #applies filter if value of filter is true
        if not filter:
            fig, ax = plt.subplots(len(self.dataName), 3, figsize=(13,3*len(self.dataName)))
            fig.suptitle('Variation of Temperature, Salinity and Oxygen Conentration with Depth')

            for ctd_id, ctd_key in enumerate(self.dataName):
                
                ctd_df = self.dataName[ctd_key]
                
                for id_col, col in enumerate(data_columns[1:]):
                    ax[ctd_id, id_col].plot(ctd_df["down"][col], ctd_df["down"]["Pressure"], '-b',
                                    ctd_df["up"][col], ctd_df["up"]["Pressure"], '-r')

                    ax[ctd_id, id_col].set(ylim=[max(ctd_df["all"]['Pressure']), 0])
                    ax[ctd_id, id_col].set(xlabel=col, ylabel=f'Pressure, Station {ctd_key}')
                    ax[ctd_id, id_col].grid()
                    ax[ctd_id, id_col].locator_params(nbins=7, axis="x")
                    ax[ctd_id, id_col].legend(["down", "up"])
                    

            fig.tight_layout()
            plt.show()
        else:
            fig, ax = plt.subplots(len(self.dataName), 3, figsize=(13, 3*len(self.dataName)))
            fig.suptitle('Variation of Temperature, Salinity and Oxygen Conentration with Depth\n(Filtered Plot)')

            for ctd_id, ctd_key in enumerate(self.dataName):
                
                ctd_df = self.dataName[ctd_key]
                
                for id_col, col in enumerate(data_columns[1:]):
                    ax[ctd_id, id_col].plot(filter_fxn(ctd_df["down"][col]), filter_fxn(ctd_df["down"]["Pressure"]), '-b',
                                    filter_fxn(ctd_df["up"][col]), filter_fxn(ctd_df["up"]["Pressure"]), '-r')
                    
                    ax[ctd_id, id_col].set(ylim=[max(ctd_df["all"]['Pressure']), 0])
                    ax[ctd_id, id_col].set(xlabel=col, ylabel=f'Pressure, Station {ctd_key}')
                    ax[ctd_id, id_col].grid()
                    ax[ctd_id, id_col].locator_params(nbins=7, axis="x")
                    ax[ctd_id, id_col].legend(["down", "up"])
                    
            fig.tight_layout()
            plt.show()

    #Plotting of Pressure against Temperature, Salinity and Oxygen (over the same range of axis)
    def variationPlotSameRange(self):
        fig, ax = plt.subplots(len(self.dataName), 3, figsize=(13,3*len(self.dataName)), sharex='col')
        fig.suptitle('Variation of Temperature, Salinity and Oxygen Conentration with Depth\n(Same Axis Range)')
        # plt.locator_params(axis='x', nbins=6)

        for ctd_id, ctd_key in enumerate(self.dataName):
            
            ctd_df = self.dataName[ctd_key]
            
            for id_col, col in enumerate(data_columns[1:]):
                ax[ctd_id, id_col].plot(ctd_df["down"][col], ctd_df["down"]["Pressure"], '-b',
                                ctd_df["up"][col], ctd_df["up"]["Pressure"], '-r')

                ax[ctd_id, id_col].set(ylim=[max(ctd_df["all"]['Pressure']), 0])
                ax[ctd_id, id_col].set(xlabel=col, ylabel=f'Pressure, Station {ctd_key}')
                ax[ctd_id, id_col].grid()
                ax[ctd_id, id_col].locator_params(nbins=7, axis="x")
                ax[ctd_id, id_col].legend(["down", "up"])

        fig.tight_layout()
        plt.show()

    #PLOT THE T-S PROFILE
    def TSPlot(self, filter):
        if not filter:
            fig, ax = plt.subplots(2, 2, figsize=( 10, 5*len(self.dataName)/2 ))
            fig.suptitle('Temperature-Salinity Diagram of Each Station')

            for ctd_id, ctd_key in enumerate(self.dataName):
                
                # Brief Manipulation to obtain the position on the subplot
                if ctd_id < 2: 
                    x_ind = 0
                    y_ind = ctd_id % 2
                else:
                    x_ind = 1
                    y_ind = ctd_id % 2
                
                ctd_df = self.dataName[ctd_key]
                
                ax[x_ind, y_ind].plot(ctd_df["down"]["Salinity"], ctd_df["down"]["Temperature"], '-b',
                                ctd_df["up"]["Salinity"], ctd_df["up"]["Temperature"], '-r')
                
                ax[x_ind, y_ind].set(xlabel="Salinity", ylabel='Temperature', title=f"Station {ctd_key}")
                ax[x_ind, y_ind].grid()
                ax[x_ind, y_ind].locator_params(nbins=7, axis="x")
                ax[x_ind, y_ind].legend(["down", "up"])


            fig.tight_layout()
            plt.show()

        else:
            fig, ax = plt.subplots(2, 2, figsize=( 10, 5*len(self.dataName)/2 ))
            fig.suptitle('Temperature-Salinity Diagram of Each Station\n(Filtered Data)')

            for ctd_id, ctd_key in enumerate(self.dataName):
                
                # Brief Manipulation to obtain the position on the subplot
                if ctd_id < 2: 
                    x_ind = 0
                    y_ind = ctd_id % 2
                else:
                    x_ind = 1
                    y_ind = ctd_id % 2
                
                ctd_df = self.dataName[ctd_key]
                
                ax[x_ind, y_ind].plot(filter_fxn(ctd_df["down"]["Salinity"]), filter_fxn(ctd_df["down"]["Temperature"]), '-b',
                                filter_fxn(ctd_df["up"]["Salinity"]), filter_fxn(ctd_df["up"]["Temperature"]), '-r')
                
                ax[x_ind, y_ind].set(xlabel="Salinity", ylabel='Temperature', title=f"Station {ctd_key}")
                ax[x_ind, y_ind].grid()
                ax[x_ind, y_ind].locator_params(nbins=7, axis="x")
                ax[x_ind, y_ind].legend(["down", "up"])


            fig.tight_layout()
            plt.show()

In [8]:
print('#################### DAY ONE ################')
dayOne = CTD('ctd_data_1', 'ctd_folder_1', 'OtherDaysData/Day1Data/ctds')
dayOne.extractData()
dayOne.variationPlot(filter=False)
dayOne.variationPlot(filter=True)
dayOne.TSPlot(filter=False)

#################### DAY ONE ################


NameError: name 'path' is not defined

In [ ]:
print('################### DAY THREE #############')
dayThree = CTD('ctd_data_3', 'ctd_folder_3', 'OtherDaysData/Day3Data/ctds')
dayThree.extractData()
dayThree.variationPlot(filter=False)
dayThree.variationPlot(filter=True)
dayThree.TSPlot(filter=False)

################### DAY THREE #############


NameError: name 'path' is not defined